In [1]:
import requests
from bs4 import BeautifulSoup as BS 
import pandas as pd
import re
import numpy as np
import time

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
ny_times = pd.read_csv('/content/gdrive/MyDrive/Cendie/NY-Times.csv')


In [26]:
ny_times.sort_values(['Date'], ascending=False)

,Date,Assigned to,Newspaper,Link,Section,Key Words,Opinion adm,Opinion CDC,Briefly observation,Abstractor,Unnamed: 10
835,9/6/2020,Eliane,The New York Times,https://www.nytimes.com/2020/09/06/health/covi...,Health,testing,Nada,Nada,New regulations require a ramp-up in testing f...,NaN,NaN
823,9/6/2020,Eliane,The New York Times,https://www.nytimes.com/2020/09/06/opinion/cor...,Opinion,"taste, meals",Nada,Nada,"Many symptoms of Covid-19 were difficult, but ...",NaN,NaN
821,9/6/2020,Eliane,The New York Times,https://www.nytimes.com/2020/09/06/at-home/cel...,At home,"Jewish community, conservative",Nada,Nada,"Because of the pandemic, conservative, reform ...",NaN,NaN
824,9/6/2020,Eliane,The New York Times,https://www.nytimes.com/2020/09/06/business/fa...,Business,retirement,Nada,Nada,Riding out the pandemic with family presents t...,NaN,NaN
825,9/6/2020,Eliane,The New York Times,https://www.nytimes.com/2020/09/06/business/ec...,Economy,people of color,Nada,Nada,Without the networks and encounters that offic...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2,1/26/2020,A,The New York Times,https://www.nytimes.com/2020/01/26/world/asia/...,Asia Pacific,Monitor passengers from China,0,0,The United States has limited passengers who h...,NaN,NaN
0,1/19/2020,A,The New York Times,https://www.nytimes.com/2020/01/18/world/asia/...,Asia Pacific,"Low transmission, unclear",Nada,0,China’s central government sought to reassure ...,"AM, NR",NaN
716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
link = ny_times['Link'].dropna()
link = link.unique()
link = link.tolist()

In [ ]:
ny_times = ny_times[ny_times['Link'].isin(link)]
ny_times.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 989 entries, 0 to 991
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Date                 989 non-null    object
 1   Assigned to          989 non-null    object
 2   Newspaper            989 non-null    object
 3   Link                 989 non-null    object
 4   Section              989 non-null    object
 5   Key Words            989 non-null    object
 6   Opinion adm          988 non-null    object
 7   Opinion CDC          988 non-null    object
 8   Briefly observation  980 non-null    object
 9   Abstractor           4 non-null      object
 10  Unnamed: 10          1 non-null      object
dtypes: object(11)
memory usage: 92.7+ KB


In [ ]:
print(link[1])

https://www.nytimes.com/2020/01/26/world/china-coronavirus.html?searchResultPosition=1


In [ ]:
response = requests.get(link[56])

In [ ]:
soup = BS(response.content)


In [ ]:
titulo = soup.find(attrs = {"data-testid":"headline"})
print(titulo.text)


In 1918, It Wasn’t the Coronavirus. It Was the Flu.


In [ ]:
titulo = soup.find(attrs = {"class":"css-rsa88z e1h9rw200"})

print(titulo.text)


AttributeError: ignored

In [ ]:
resumen = soup.find(attrs = {"class":"css-w6ymp8 e1wiw3jv0"})
print(resumen.text)


Americans have been alarmed by empty grocery shelves, but while food suppliers and retailers say they are struggling with surging demand, they insist the supply chain remains strong.


In [ ]:
fecha = soup.find("time")["datetime"]
print(fecha)

2020-03-12T22:31:45-04:00


In [ ]:
fecha = soup.find(attrs = {"class": "css-75y64v e16638kd2"})["datetime"]
print(fecha)

In [ ]:
cuerpo = soup.find(attrs = {"class": "meteredContent css-1r7ky0e"})
cuerpo = cuerpo.find_all(attrs = {"class": "css-axufdj evys1bk0"})
print(cuerpo)

[<p class="css-axufdj evys1bk0">The aisles and aisles of empty store shelves give the appearance that the United States, improbably and alarmingly, is running out of food.</p>, <p class="css-axufdj evys1bk0">But the nation’s biggest retailers, dairy farmers and meat producers say that isn’t so. The food supply chain, they say, remains intact and has been ramping up to meet the unprecedented stockpiling brought on by the coronavirus pandemic.</p>, <p class="css-axufdj evys1bk0">Even so, shoppers can most likely expect to see empty shelves intermittently, as the nation’s network of food producers, distributors and retailers are stretched as never before. Industries that are calibrated to supply consumers with just enough of what they need on a given day <a class="css-1g7m0tk" href="https://www.nytimes.com/2020/03/21/business/coronavirus-freezers-sold-out.html" title="">cannot keep up with a nationwide surge</a> of relentless shopping fueled in large part by fear.</p>, <p class="css-axufd

In [ ]:
texts = []
for p in cuerpo:
  texts.append(p.text)

body_text = '\n'.join(texts)

print(body_text)

The aisles and aisles of empty store shelves give the appearance that the United States, improbably and alarmingly, is running out of food.
But the nation’s biggest retailers, dairy farmers and meat producers say that isn’t so. The food supply chain, they say, remains intact and has been ramping up to meet the unprecedented stockpiling brought on by the coronavirus pandemic.
Even so, shoppers can most likely expect to see empty shelves intermittently, as the nation’s network of food producers, distributors and retailers are stretched as never before. Industries that are calibrated to supply consumers with just enough of what they need on a given day cannot keep up with a nationwide surge of relentless shopping fueled in large part by fear.
Food suppliers and retailers are now not only struggling to satiate crushing demand for canned soup and oat milk, they are battling a perception that the scary scenes at the grocery store reflect a fundamental breakdown.
“There is food being produced

In [ ]:
df_notas_times = pd.DataFrame(columns = ['Titulo' , 'Resumen' , 'Url', 'Fecha', 'Texto'])


In [ ]:
df_notas_times = df_notas_times.append({'Titulo': titulo.text, 'Resumen': resumen.text ,'Url': link[900] , 'Fecha': fecha, 'Texto': body_text}, ignore_index=True)

In [ ]:
df_notas_times

,Titulo,Resumen,Url,Fecha,Texto
0,‘There Is Plenty of Food in the Country’,[Americans have been alarmed by empty grocery ...,https://www.nytimes.com/2020/10/04/nyregion/ny...,2020-03-21T19:20:09-04:00,The aisles and aisles of empty store shelves g...
1,‘There Is Plenty of Food in the Country’,Americans have been alarmed by empty grocery s...,https://www.nytimes.com/2020/10/04/nyregion/ny...,2020-03-21T19:20:09-04:00,The aisles and aisles of empty store shelves g...


In [ ]:
def scraper_nypost(csv_ny_times_pad):

  ny_post = pd.read_csv(csv_ny_times_pad)
  link = ny_post['Link'].dropna()
  link = link.unique()
  link = link.tolist()

  df_notas_times = pd.DataFrame(columns = ['Titulo' ,'Url', 'Fecha', 'Texto'])

  error_titulo = []
  error_fecha = []
  error_cuerpo = []

  for i in range(len(link)):


    print(i)

    time.sleep(1 * np.random.random())
    response = requests.get(link[i])
    soup = BS(response.content)
    try:
      titulo = soup.find(attrs = {"data-testid":"headline"})
      titulo = titulo.text
    except:
      print(f'Error titulo: {i}')
      error_titulo.append(link[i])

    # try:
    #   resumen = soup.find(attrs = {"class":"css-w6ymp8 e1wiw3jv0"})
    # except:
    #   pass
    try:
      fecha = soup.find("time")["datetime"]
    except:
      print(f'Error fecha: {i}')
      error_fecha.append(link[i])

    try:
      cuerpo = soup.find(attrs = {"class": "meteredContent css-1r7ky0e"})
      cuerpo = cuerpo.find_all(attrs = {"class": "css-axufdj evys1bk0"})
      texts = []
      for p in cuerpo:
        
        texts.append(p.text)

      body_text = '\n'.join(texts)
    except:
      print(f'Error cuerpo:{i}')
      error_cuerpo.append(link[i])

    df_notas_times = df_notas_times.append({'Titulo': titulo, 'Url': link[i] , 'Fecha': fecha, 'Texto': body_text}, ignore_index=True)
  

  return df_notas_times , error_titulo, error_fecha, error_cuerpo

In [ ]:
ny_times = scraper_nypost('/content/gdrive/MyDrive/Cendie/NY-Times.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
Error fecha: 36
Error cuerpo:36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
Error cuerpo:79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
Error cuerpo:128
129
130
Error cuerpo:130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
Error titulo: 185
186
187
188
189
Error fecha: 189
Error cuerpo:189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
Error cuerpo:221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
Err

In [ ]:
ny_times = ny_times[0]

In [ ]:
from google.colab import files

ny_times.to_csv('ny_times.csv')
files.download('ny_times.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>